# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
from scipy.stats import linregress
import numpy as np
import seaborn as sns
import pandas as pd
import requests
import gmaps
gmaps.configure()


# Import API key
from api_keys import geoapify_key
gmaps.configure(api_key=geoapify_key)

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,albany,42.6001,-73.9662,57.79,44,1,12.66,US,1667857658
1,1,mayo,38.8876,-76.5119,78.75,45,0,8.05,US,1667857815
2,2,dunedin,-45.8742,170.5036,59.38,64,60,4.12,NZ,1667857661
3,3,kavieng,-2.5744,150.7967,82.24,75,92,4.63,PG,1667857753
4,4,jamestown,42.0970,-79.2353,49.71,52,0,14.97,US,1667857816


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
locations = city_data_df[['Lat', 'Lng']]
locations

humidity = city_data_df["Humidity"].astype(float)

In [4]:
#Define the size of the map (I couldn't use HVplots bc it wasnt downloading correctly and throwing 
#erros related to the )
fig = gmaps.figure(layout={
        'width': '400px',
        'height': '600px',
        'padding': '3px',
        'border': '1px solid black'
})
#add markers 
markers = gmaps.marker_layer(locations)

#create/add heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=10,
                                point_radius=1)
heat_layer.gradient =['blue', 'gray', 'yellow', 'red']
heat_layer.opacity = 0.2
fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='3px', width='400px'))

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
NarrowedDF = city_data_df.loc[(city_data_df['Max Temp'] < 70) & (city_data_df['Country'] == 'US') & (city_data_df['Cloudiness'] < 5)]
NarrowedDF


# Display sample data
# YOUR CODE HERE

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,albany,42.6001,-73.9662,57.79,44,1,12.66,US,1667857658
4,4,jamestown,42.0970,-79.2353,49.71,52,0,14.97,US,1667857816
14,14,storm lake,42.6411,-95.2097,45.64,36,0,14.97,US,1667857820
155,155,hamilton,39.1834,-84.5333,65.64,24,0,9.22,US,1667857823
158,158,sitka,57.0531,-135.3300,35.49,25,0,7.05,US,1667857868
188,188,rock springs,41.5875,-109.2029,53.04,46,0,23.02,US,1667857715
191,191,gillette,44.2911,-105.5022,47.32,49,0,26.46,US,1667857883
305,305,nantucket,41.2835,-70.0995,68.14,90,0,13.80,US,1667857887
379,379,grafton,42.2070,-71.6856,69.80,33,0,13.80,US,1667857949
415,415,waverly,42.7392,-84.6208,51.96,45,0,10.36,US,1667857960


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity 
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df = NarrowedDF.drop(['City_ID','Max Temp', 'Cloudiness', 'Wind Speed', 'Date'], axis=1)
hotel_df['Hotel Name'] = np.nan
hotel_df


,City,Lat,Lng,Humidity,Country,Hotel Name
0,albany,42.6001,-73.9662,44,US,NaN
4,jamestown,42.0970,-79.2353,52,US,NaN
14,storm lake,42.6411,-95.2097,36,US,NaN
155,hamilton,39.1834,-84.5333,24,US,NaN
158,sitka,57.0531,-135.3300,25,US,NaN
188,rock springs,41.5875,-109.2029,46,US,NaN
191,gillette,44.2911,-105.5022,49,US,NaN
305,nantucket,41.2835,-70.0995,90,US,NaN
379,grafton,42.2070,-71.6856,33,US,NaN
415,waverly,42.7392,-84.6208,45,US,NaN


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
from api_keys import second_key
radius = 10000
params = {
    "categories": 'accommodation.hotel',
    "apiKey": second_key,
    "limit": 20

}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"]= f'circle:{lng},{lat},{radius}'
    params["bias"]= f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"


    # Make and API request using the params dictionaty
    response_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
albany - nearest hotel: No hotel found
jamestown - nearest hotel: DoubleTree Jamestown
storm lake - nearest hotel: Super 8 Storm Lake
hamilton - nearest hotel: North Vista Manor
sitka - nearest hotel: Aspen Suites Hotel
rock springs - nearest hotel: Sands Inn
gillette - nearest hotel: Montgomery Hotel
nantucket - nearest hotel: No hotel found
grafton - nearest hotel: Grafton Inn
waverly - nearest hotel: Crowne Plaza Lansing West
worthington - nearest hotel: Magnuson Grand Columbus North Hotel
saint anthony - nearest hotel: Delta Hotels by Marriott


,City,Lat,Lng,Humidity,Country,Hotel Name
0,albany,42.6001,-73.9662,44,US,No hotel found
4,jamestown,42.0970,-79.2353,52,US,DoubleTree Jamestown
14,storm lake,42.6411,-95.2097,36,US,Super 8 Storm Lake
155,hamilton,39.1834,-84.5333,24,US,North Vista Manor
158,sitka,57.0531,-135.3300,25,US,Aspen Suites Hotel
188,rock springs,41.5875,-109.2029,46,US,Sands Inn
191,gillette,44.2911,-105.5022,49,US,Montgomery Hotel
305,nantucket,41.2835,-70.0995,90,US,No hotel found
379,grafton,42.2070,-71.6856,33,US,Grafton Inn
415,waverly,42.7392,-84.6208,45,US,Crowne Plaza Lansing West


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display
Hlocations = hotel_df[['Lat', 'Lng']]
Hlocations

Hnames = hotel_df.iloc[:,5]
Cnames = hotel_df.iloc[:,4]

hotel_df['Hover Text'] = f'Hotel found: ' + hotel_df["Hotel Name"] + f'; Country: ' + hotel_df['Country']
hotel_df

,City,Lat,Lng,Humidity,Country,Hotel Name,Hover Text
0,albany,42.6001,-73.9662,44,US,No hotel found,Hotel found: No hotel found; Country: US
4,jamestown,42.0970,-79.2353,52,US,DoubleTree Jamestown,Hotel found: DoubleTree Jamestown; Country: US
14,storm lake,42.6411,-95.2097,36,US,Super 8 Storm Lake,Hotel found: Super 8 Storm Lake; Country: US
155,hamilton,39.1834,-84.5333,24,US,North Vista Manor,Hotel found: North Vista Manor; Country: US
158,sitka,57.0531,-135.3300,25,US,Aspen Suites Hotel,Hotel found: Aspen Suites Hotel; Country: US
188,rock springs,41.5875,-109.2029,46,US,Sands Inn,Hotel found: Sands Inn; Country: US
191,gillette,44.2911,-105.5022,49,US,Montgomery Hotel,Hotel found: Montgomery Hotel; Country: US
305,nantucket,41.2835,-70.0995,90,US,No hotel found,Hotel found: No hotel found; Country: US
379,grafton,42.2070,-71.6856,33,US,Grafton Inn,Hotel found: Grafton Inn; Country: US
415,waverly,42.7392,-84.6208,45,US,Crowne Plaza Lansing West,Hotel found: Crowne Plaza Lansing West; Countr...


In [39]:
%%capture --no-display 

HT = f'{hotel_df.iloc[0:12,6]}'

markers = gmaps.marker_layer(Hlocations)

# Display the map
fig = gmaps.figure(layout={
        'width': '400px',
        'height': '600px',
        'padding': '3px',
        'border': '1px solid black'
})

#create/add marker layer
Hname_layer = gmaps.symbol_layer(Hlocations, hover_text=HT)
fig.add_layer(Hname_layer)

# Hname_layer = gmaps.symbol_layer(Hlocations, hover_text=cc)
# fig.add_layer(Hname_layer)

fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='3px', width='400px'))